In [1]:
# %matplotlib ipympl

import os
import glob
import torch
import tqdm
import time
import csv
from torch import nn
from torch.utils.data import DataLoader, ConcatDataset, random_split
# from learning.model.nodevalue import DWNodeValueModel
from learning.data.process_utils import move_to_device
from learning.data.nv_utils import (
    extract_input, extract_output_target, collate_nv_data
)
from learning.data.nv_dataset import NodeValueDataset
from learning.model.nodevalue import NodeValueModel

In [2]:
history_folder = "server/history/"
history_files = sorted(glob.glob(os.path.join(history_folder, "history_*.json")))

datasets = [NodeValueDataset(f) for f in history_files]
concatenated_dataset = ConcatDataset(datasets)

train_ds, val_ds, test_ds = random_split(
    concatenated_dataset,
    [0.8, 0.1, 0.1],
)

In [3]:
# Set a fixed seed for reproducibility
seed = 42
torch.manual_seed(seed)
generator = torch.Generator().manual_seed(seed)

train_loader = DataLoader(train_ds, 10000, pin_memory=True, shuffle=True, 
                         collate_fn=collate_nv_data, generator=generator)
val_loader = DataLoader(val_ds, 20000, pin_memory=True, shuffle=True, 
                       collate_fn=collate_nv_data, generator=generator)

print("train_ds", len(train_ds))
print("val_ds", len(val_ds))
print("test_ds", len(test_ds))

train_ds 578341
val_ds 72292
test_ds 72292


In [4]:
batch = next(iter(train_loader))

In [5]:
torch.bincount(batch["winners"])

tensor([1311, 1260, 1218, 1247, 1259, 1198, 1248, 1259])

In [6]:
perm = torch.tensor([7,6,5,4,3,2,1,0])
perm[[0,0,1,1,2,2,3,3,4,4,5,5,6,6,7,7]]

tensor([7, 7, 6, 6, 5, 5, 4, 4, 3, 3, 2, 2, 1, 1, 0, 0])

In [7]:
def calculate_mean(train_loader):
    # Calculate statistics properly with variable-sized batches
    sum_of_values = 0.0
    count_of_values = 0
    feature_sums = None
    batch_count = 0

    for batch in tqdm.tqdm(train_loader, desc="Calculating average node states"):
        current_states = batch["nodes_states"]
        
        # Add to running statistics
        sum_of_values += current_states.sum().item()
        count_of_values += current_states.shape[0]
        
        # For per-feature statistics
        if feature_sums is None:
            # Initialize with zeros matching the feature dimension
            feature_sums = torch.zeros(current_states.shape[1], dtype=torch.float32)
        
        # Sum across batch dimension for feature-wise statistics
        feature_sums += current_states.sum(dim=0)
        
        batch_count += 1

    # Calculate the averages
    feature_averages = feature_sums / count_of_values 

    print(f"Statistics calculated over {batch_count} batches")
    print(f"Shape of feature dimension: {feature_sums.shape}")
    print(f"Feature-wise averages:")
    for i, avg in enumerate(feature_averages):
        print(f"  Feature {i}: {avg.item():.6f}")
    return feature_averages

def calculate_std(train_loader, feature_averages):
    # Calculate statistics for standard deviation with variable-sized batches
    sum_squared_diff = 0.0
    count_of_values = 0
    feature_sum_squared_diff = None
    batch_count = 0

    for batch in tqdm.tqdm(train_loader, desc="Calculating std dev of node states"):
        current_states = batch["nodes_states"]
        
        # Add to running statistics
        sum_squared_diff += ((current_states - feature_averages) ** 2).sum().item()
        count_of_values += current_states.shape[0]
        
        # For per-feature statistics
        if feature_sum_squared_diff is None:
            # Initialize with zeros matching the feature dimension
            feature_sum_squared_diff = torch.zeros(current_states.shape[1], dtype=torch.float32)
        
        # Sum squared differences across batch dimension for feature-wise statistics
        feature_sum_squared_diff += ((current_states - feature_averages) ** 2).sum(dim=0)
        
        batch_count += 1

    # Calculate the standard deviations
    overall_std = torch.sqrt(torch.tensor(sum_squared_diff / count_of_values))
    feature_stds = torch.sqrt(feature_sum_squared_diff / count_of_values)

    print(f"Standard deviation statistics calculated over {batch_count} batches")
    print(f"Overall standard deviation: {overall_std.item():.6f}")
    print(f"Feature-wise standard deviations:")
    for i, std in enumerate(feature_stds):
        print(f"  Feature {i}: {std.item():.6f}")
    return feature_stds

In [8]:
model_state = None
optim_state = None
latest_epoch = -1

with_dice_scatter = True

model_name = "nodevalue" + ("_no_dice" if not with_dice_scatter else "")
models_dir = f"learning/{model_name}_checkpoints" 

os.makedirs(models_dir, exist_ok=True)

# Find the latest checkpoint file (with highest epoch number)
checkpoint_files = glob.glob(os.path.join(models_dir, f"{model_name}_*.pt"))
if checkpoint_files:
    # Extract epoch numbers from filenames
    epoch_nums = [int(f.split("_")[-1].split(".")[0]) for f in checkpoint_files]
    latest_epoch = max(epoch_nums)
    latest_checkpoint = os.path.join(models_dir, f"{model_name}_{latest_epoch:06}.pt")
    print(f"Loading latest checkpoint: {latest_checkpoint} (epoch {latest_epoch})")
    checkpoint = torch.load(latest_checkpoint, map_location=torch.device('cpu'))
    model_state = checkpoint["model_state"]
    optim_state = checkpoint["optim_state"]
else:
    print("No valid checkpoint files found")
    
start_epoch = latest_epoch + 1

Loading latest checkpoint: learning/nodevalue_checkpoints/nodevalue_000699.pt (epoch 699)


In [9]:
node_value_model = NodeValueModel(with_dice_scatter=with_dice_scatter)
num_params = sum(p.numel() for p in node_value_model.parameters() if p.requires_grad)
print(f"Model created. Total trainable parameters: {num_params:,}")

Model created. Total trainable parameters: 4,244


In [ ]:
# Load the model before everything else so that I can freeze some layers
if model_state is not None:
    node_value_model.load_state_dict(model_state)
    print(f"Model state loaded.")

# node_value_model.gat_layers[0].requires_grad_(False)
# node_value_model.gat_layers[1].passthrough_coef.requires_grad_(False)
# node_value_model.gat_layers[2].requires_grad_(False)

# node_value_model.gat_layers[1].reset_parameters() 
# nn.init.constant_(node_value_model.gat_layers[1].passthrough_coef, -2)

# node_value_model.gat_layers[2].reset_parameters() 
# nn.init.constant_(node_value_model.gat_layers[2].passthrough_coef, -1)

Model state loaded.


Parameter containing:
tensor(-2., requires_grad=True)

In [ ]:
# ── CSV set‑up: log one row per epoch ───────────────────────────────────────
csv_path = f"{model_name}_epoch_metrics.csv"
epoch_log_file_exists = os.path.isfile(csv_path)
if epoch_log_file_exists and os.path.getsize(csv_path) == 0:
    os.remove(csv_path)
    epoch_log_file_exists = False

csv_file = open(csv_path, "a", newline="")  # Open in append mode
csv_writer = csv.writer(csv_file)

# Only write header if file doesn't exist yet
if not epoch_log_file_exists:
    csv_writer.writerow([
        "epoch",           # 0‑based epoch index
        "train_loss",      # average training loss for the epoch
        "val_loss",        # average validation loss for the epoch
        "train_time_sec",  # seconds spent in training phase
        "val_time_sec",    # seconds spent in validation phase
        "total_time_sec"   # train + val
    ])

# ── model / optimizer prep ─────────────────────────────────────────────────

n_epochs   = 10000
device     = torch.device("cuda", 0)
train_time_limit = 10 * 60
val_time_limit = train_time_limit / 5
criterion  = nn.CrossEntropyLoss()

node_value_model = node_value_model.to(device)

optimizer  = torch.optim.Adam(
    [p for p in node_value_model.parameters() if p.requires_grad],
    lr=1e-2
)
reset_optimizer = True
if not reset_optimizer and optim_state is not None:
    optimizer.load_state_dict(optim_state)

scheduler  = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min",
    factor=0.25,
    patience=10,
    threshold=0.0001,
    min_lr=1e-8
)


In [12]:
# ── INITIAL (untrained) LOSS EVALUATION ───────────────────────────────────────
if not epoch_log_file_exists:
    node_value_model.eval()
    with torch.no_grad():
        t_start = time.time()

        # avg loss on training set *without* gradient tracking
        init_train_sum, init_train_batches = 0.0, 0
        for b in tqdm.tqdm(train_loader, desc="init‑train"):
            b = move_to_device(b, device)
            out = node_value_model(*extract_input(b, with_dice_scatter=with_dice_scatter))
            o, t = extract_output_target(b, out)
            init_train_sum += criterion(o, t).item()
            init_train_batches += 1
            if (time.time() - t_start) > train_time_limit: 
                print(f"Stopping training after >{train_time_limit} seconds.")
                break
        init_train_loss = init_train_sum / init_train_batches
        t_train_done = time.time()

        # avg loss on validation set
        init_val_sum, init_val_batches = 0.0, 0
        for vb in tqdm.tqdm(val_loader, desc="init‑val"):
            vb = move_to_device(vb, device)
            vout = node_value_model(*extract_input(vb, with_dice_scatter=with_dice_scatter))
            vo, vt = extract_output_target(
                vb, vout
            )
            init_val_sum += criterion(vo, vt).item()
            init_val_batches += 1
            if (time.time() - t_train_done) > val_time_limit:
                print(f"Stopping validation after >{val_time_limit} seconds.")
                break
        init_val_loss = init_val_sum / init_val_batches
        t_val_done = time.time()

    # times
    init_train_time = t_train_done - t_start
    init_val_time   = t_val_done - t_train_done
    init_total_time = t_val_done - t_start

    # write initial row (epoch = None)
    csv_writer.writerow([
        -1,
        init_train_loss,
        init_val_loss,
        init_train_time,
        init_val_time,
        init_total_time
    ])
    csv_file.flush()
    print(
        f"Initial (-1) | "
        f"Train {init_train_loss:.4f} | "
        f"Val {init_val_loss:.4f} | "
        f"Time {init_total_time:.1f}s (T {init_train_time:.1f}s | "
        f"V {init_val_time:.1f}s)"
    )

In [13]:
scheduler.get_last_lr()

[0.01]

In [14]:
for epoch in range(start_epoch, n_epochs):
    t_start = time.time()

    # ── TRAINING -----------------------------------------------------------
    node_value_model.train()
    sum_train_loss = torch.tensor(0.0, device=device)
    n_train_batches = 0

    t_batch_tqdm = tqdm.tqdm(
        train_loader, 
        desc=f"train {epoch}" 
    )
    for t_batch in t_batch_tqdm:
        t_batch = move_to_device(t_batch, device)

        optimizer.zero_grad()

        model_out = node_value_model(*extract_input(t_batch, with_dice_scatter=with_dice_scatter))
        outputs, targets = extract_output_target(t_batch, model_out)
        loss = criterion(outputs, targets)
        
        loss.backward()
        optimizer.step()
    
        if torch.isnan(loss) or torch.isinf(loss):
            raise ValueError(f"Loss contains NaN or Inf values")
        
        sum_train_loss += loss.detach()
        n_train_batches += 1
        t_batch_tqdm.set_postfix(av_loss=sum_train_loss.item()/n_train_batches)
        if (time.time() - t_start) > train_time_limit: 
            print(f"Stopping training after >{train_time_limit} seconds.")
            break
        
    t_batch_tqdm.close()

    avg_train_loss = sum_train_loss.item() / n_train_batches
    t_train_done = time.time()

    # ── VALIDATION ---------------------------------------------------------
    node_value_model.eval()
    sum_val_loss = 0.0
    n_val_batches = 0
    with torch.no_grad():
        v_batch_tqdm = tqdm.tqdm(
            val_loader, 
            desc=f"val {epoch}" 
        )
        for v_batch in v_batch_tqdm:
            v_batch = move_to_device(v_batch, device)

            v_out = node_value_model(*extract_input(v_batch, with_dice_scatter=with_dice_scatter))
            v_outputs, v_targets = extract_output_target(v_batch, v_out)
            val_loss = criterion(v_outputs, v_targets)

            if torch.isnan(val_loss) or torch.isinf(val_loss):
                raise ValueError(f"Loss contains NaN or Inf values")
            
            sum_val_loss += val_loss
            n_val_batches += 1
            v_batch_tqdm.set_postfix(av_loss=sum_val_loss.item()/n_val_batches)
            if (time.time() - t_train_done) > val_time_limit:
                print(f"Stopping validation after >{val_time_limit} seconds.")
                break

        v_batch_tqdm.close()

    avg_val_loss = sum_val_loss.item() / n_val_batches
    t_val_done = time.time()

    # Step the learning rate scheduler after each epoch
    scheduler.step(avg_val_loss)

    # ── CSV logging ---------------------------------------------------------
    train_time = t_train_done - t_start
    val_time   = t_val_done   - t_train_done
    total_time = t_val_done   - t_start


    checkpoint_data = {
        "epoch": epoch,
        "optim_state": optimizer.state_dict(),
        "model_state": node_value_model.state_dict()
    }
    torch.save(checkpoint_data, os.path.join(models_dir, f"{model_name}_{epoch:06}.pt"))
    
    csv_writer.writerow([
        epoch,
        avg_train_loss,
        avg_val_loss,
        train_time,
        val_time,
        total_time
    ])
    csv_file.flush()  # ensure data is written even if run aborts

    # ── console printout ----------------------------------------------------
    print(
        f"Epoch {epoch} | "
        f"Train {avg_train_loss:.4f} | "
        f"Val {avg_val_loss:.4f} | "
        f"LR {scheduler.get_last_lr()[0]:.6f}"
    )
# ── tidy‑up ----------------------------------------------------------------
csv_file.close()

train 700:   0%|          | 0/58 [00:00<?, ?it/s]

/home/maxim/Programming/dicewar_learning/learning/model/utils.py:53: UserWarning: index_reduce() is in beta and the API may change at any time. (Triggered internally at /pytorch/aten/src/ATen/native/cuda/Indexing.cu:1432.)
  max_per_index.index_reduce_(
val 700: 100%|██████████| 4/4 [00:06<00:00,  1.71s/it, av_loss=0.871]


Epoch 700 | Train 1.0426 | Val 0.8709 | LR 0.010000


val 701: 100%|██████████| 4/4 [00:06<00:00,  1.66s/it, av_loss=0.752]


Epoch 701 | Train 0.8508 | Val 0.7518 | LR 0.010000


...

Epoch 997 | Train 0.6622 | Val 0.5670 | LR 0.000000


val 998: 100%|██████████| 4/4 [00:07<00:00,  1.86s/it, av_loss=0.567]


Epoch 998 | Train 0.6629 | Val 0.5666 | LR 0.000000


val 999: 100%|██████████| 4/4 [00:07<00:00,  1.89s/it, av_loss=0.565]

Epoch 999 | Train 0.6628 | Val 0.5653 | LR 0.000000


In [15]:
outputs, targets

(tensor([[ 2.2256,  0.2028,  5.3070,  ...,  0.0000,  0.0162,  0.4127],
         [ 1.8853, -0.0358,  0.3338,  ...,  1.9942,  0.0000,  1.9039],
         [ 0.0000,  0.0000,  0.4560,  ...,  1.0192,  0.6509,  1.7127],
         ...,
         [ 3.7650,  0.0000, -0.1808,  ...,  5.6089,  3.4821,  0.4775],
         [-0.6968, -0.1935,  1.1592,  ...,  0.0000,  0.2794,  2.3095],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  1.0781,  6.7716]],
        device='cuda:0', grad_fn=<IndexPutBackward0>),
 tensor([2, 5, 3,  ..., 5, 7, 4], device='cuda:0'))